## FHIR Implementation Guide Testing Pipeline
This notebook provides a comprehensive pipeline for automatically extracting requirements from FHIR Implementation Guides and generating executable test suites. The pipeline transforms Implementation Guide (IG) documentation into structured test code that can validate FHIR server implementations.

#### Overview
This automated pipeline takes FHIR Implementation Guide documentation and produces comprehensive test suites through several integrated stages:

- Implementation Guide Preparation: Convert and clean IG HTML documentation to markdown format
- Requirements Extraction: Use AI to identify and extract testable requirements from the IG
- Requirements Refinement: Consolidate and refine the extracted requirements
- Requirements Downselection: Combine multiple requirement sets and remove duplicates
- Test Plan Generation: Convert requirements into detailed test specifications
- Test Kit Generation: Generate executable Inferno test code

#### Running this Notebook
The notebook is structured to run each stage sequentially. You can either:

- Run the complete pipeline: Execute all cells to process a complete IG
- Run individual stages: Execute specific sections as needed

Inputs and output directories can be customized for each step. The pipeline automatically saves intermediate outputs in checkpoint directories for review and iteration.

#### Output Structure
The pipeline generates organized outputs in checkpoint directories:

checkpoints/

├── markdown1/          # Converted markdown files

├── markdown2/          # Cleaned markdown files  

├── requirements_extraction/   # Initial AI-extracted requirements

├── revised_reqs_extraction/  # Refined requirements lists

├── requirements_downselect/  # Final consolidated requirements

├── testplan_generation/     # Detailed test specifications

└── testkit_generation/      # Executable Inferno test suites

Each stage preserves its outputs, allowing for iteration, review, and alternative processing paths.

## Setup

#### Importing Notebooks as Modules (from the [Jupyter Notebook Documentation](https://jupyter-notebook.readthedocs.io/en/4.x/examples/Notebook/rstversions/Importing%20Notebooks.html))

In [5]:
import inspect
import json
import llm_utils
import importlib
import tiktoken
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from glob import glob

## Initializing LLM Clients

In [6]:
importlib.reload(llm_utils)
llm_clients = llm_utils.LLMApiClient()

In [7]:
llm_clients.clients

{'claude': <anthropic.Anthropic at 0x16907a9f0>,
 'gemini': genai.GenerativeModel(
     model_name='models/gemini-2.5-pro',
     generation_config={'max_output_tokens': 8192, 'temperature': 0.3},
     safety_settings={<HarmCategory.HARM_CATEGORY_HARASSMENT: 7>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HATE_SPEECH: 8>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 9>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 10>: <HarmBlockThreshold.BLOCK_NONE: 4>},
     tools=None,
     system_instruction=None,
     cached_content=None
 ),
 'gpt': <openai.OpenAI at 0x10ce50d10>}

## Implementation Guide Preparation

### Stage 1: Text Extraction and Cleaning
- Converts HTML IG files to markdown format
- Cleans unnecessary content (navigation, headers, formatting artifacts)
- Prepares clean, structured text for AI processing

Inputs: HTML files from FHIR IG downloads

Outputs: Clean markdown files

#### 1a) HTML to Markdown Conversion

In [ ]:
import html_narrative_extractor #import html extractor module

# Process directory with default settings
result = html_narrative_extractor.convert_local_html_to_markdown(
    "../us-core/test_set", #input directory of downloaded IG HTML files
    "checkpoints/markdown1/" #output directory
)

Found 1 HTML files to process
Processed 1/1 files
Conversion complete. Successfully processed 1 files. Encountered 0 errors.


#### 1b) Markdown Post-processing

In [ ]:
import markdown_cleaner #import markdown cleaner module
markdown_cleaner.process_directory("checkpoints/markdown1", #input directory of IG markdown files
                                   "checkpoints/markdown2/") #output directory

Found 1 markdown files in checkpoints/markdown1_test
Cleaned and saved: checkpoints/markdown2_test/CapabilityStatement-us-core-server.md

Processing complete: 1 files successfully cleaned, 0 failed


## Stage 2: Requirements Extraction

### 2a) Prompt-based Requirement Extraction
LLM Requirements Identification
- Processes markdown files using LLM to extract clear, testable requirements
- Formats requirements according to set standards, following INCOSE guidance
- Generates structured requirements with IDs, descriptions, actors, and conformance levels
- Handles large documents through chunking
- Provides source tracking

Inputs: Cleaned IG markdown files

Outputs: Structured requirements list as markdown file

In [29]:
import reqs_extraction #import LLM requirements extraction module

In [30]:
reqs_extraction.run_requirements_extractor(
    'checkpoints/markdown2_test', #input directory of markdown files
    'checkpoints/requirements_extraction/', #output directory
    'claude', #set API type
    llm_clients) #initialize llm clients


Processing Implementation Guide with Claude...
This may take several minutes depending on the size of the Implementation Guide.

Processing complete!
Generated requirements document: checkpoints/requirements_extraction/claude_reqs_list_v1_20250829_071313.md
Processed 1 files


### 2b) Requirements Refinement
LLM-Based Requirements Review & Consolidation
- Filters and identifies only testable requirements from raw extractions
- Consolidates duplicate requirements and merges related ones
- Applies consistent formatting and structure
- Removes non-testable assertions and narrative content

Inputs: Raw requirements from extraction stage in markdown format

Outputs: Refined requirements list in markdown format

In [31]:
# import requirements refinement script as module
import reqs_reviewer
importlib.reload(reqs_reviewer)

<module 'reqs_reviewer' from '/Users/ceadams/Documents/onclaive/onclaive/pipeline/reqs_reviewer.py'>

### Large Number of Requirements (500+)

In [32]:
result = reqs_reviewer.run_batch_requirements_refinement(
    input_file="/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/requirements_extraction/claude_reqs_list_v1_20250829_071313.md", #input requirements list markdown file
    llm_client_instance=llm_clients,  #initialize llm clients
    batch_size=75,  #set batch size
    api_type="claude"  #set API type
)

STARTING BATCH PROCESSING
Input: /Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/requirements_extraction/claude_reqs_list_v1_20250829_071313.md
Output: checkpoints/revised_reqs_extraction
Batch size: 75 requirements
API: claude

File size: 32,242 characters
Splitting requirements...
Found 65 total requirements
Will process in 1 batches

BATCH 1/1
   Requirements: 65 (#1-#65)
   Size: 32,242 chars (~8,060 tokens)
   Completed in 98.4s
   Progress: 1/1 (100.0%)
   ETA: 0.0 minutes remaining

COMBINING RESULTS
--------------------
Merging batch results and renumbering...
   Processing batch 1 results...
   Renumbered 65 requirements
BATCH PROCESSING COMPLETE!
Output saved: checkpoints/revised_reqs_extraction/claude_refined_requirements_20250829_071642.md
Original requirements: 65
Final requirements: 65
Successful batches: 1/1
Failed batches: 0/1
Total time: 1.6 minutes
Average per batch: 98.4 seconds


### Small Number of Requirements (<500)

In [14]:
# Manual refinement with specific file
refinement_result = reqs_reviewer.refine_requirements(
    input_file="checkpoints/requirements_extraction/us-core/claude_reqs_list_v1_20250826_084443.md",
    api_type="claude",
    output_dir="checkpoints/revised_reqs_extraction",
    llm_client_instance=llm_clients
)

print(f"✅ Refined requirements saved to: {refinement_result['output_file']}")
print(f"📊 {refinement_result['original_requirements_count']} → {refinement_result['requirements_count']} requirements")

if refinement_result.get('warnings'):
    print("⚠️  Warnings:")
    for warning in refinement_result['warnings']:
        print(f"  - {warning}")

INFO:root:Prompt environment set up at: /Users/ceadams/Documents/onclaive/onclaive/prompts
INFO:root:Starting requirements refinement with claude
INFO:root:Original requirements count: 67
INFO:root:Input size: 30785 characters, ~7696 tokens
INFO:root:Sending requirements to claude for refinement...
INFO:root:Estimated input tokens: 8307
INFO:root:API input limit: 180000
INFO:root:API output limit: 8192
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:root:Requirements refinement complete. Output saved to: checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250826_085415.md
INFO:root:Refined 67 -> 66 requirements
INFO:root:Output size: 29582 characters, ~7395 tokens


✅ Refined requirements saved to: checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250826_085415.md
📊 67 → 66 requirements


### 2c) Requirements Downselection
- Combines multiple requirements lists from different extraction runs
- Uses semantic similarity analysis to identify and remove duplicates
- Creates a deduplicated final requirements set

Inputs: Multiple refined requirements files in markdown or JSON format
Outputs: Final consolidated requirements in markdown or JSON format

In [ ]:
import requirement_downselect
importlib.reload(requirement_downselect)
requirement_downselect.full_pass(
    md_files=['',''] #enter list of markdown files to collate from
    )

/Users/ceadams/Documents/onclaive/onclaive/pipeline/requirement_downselect.py:23: SyntaxWarning: invalid escape sequence '\*'
  headings = list(re.finditer("\*\*\w+\*\*\:", split))
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-mpnet-base-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Final number of requirements: 62
Output saved in Markdown format in directory: checkpoints/requirements_downselect


In [ ]:
import requirement_downselect
importlib.reload(requirement_downselect)
requirement_downselect.full_pass(
    md_files=get_md_files_from_directory("checkpoints/requirements_extraction/")
)

## Stage 3: Test Plan Generation
- Transforms requirements into detailed test specifications
- Analyzes IG capability statements for context
- Generates implementation strategies with specific FHIR operations
- Creates structured test plans with validation criteria

Inputs: Refined requirements and IG capability statements in markdown format

Outputs: Detailed test plan in markdown format

In [21]:
import logging
llm_clients.logger.setLevel(logging.INFO)

##### Without RAG- Faster

In [25]:
import req_to_testplan
importlib.reload(req_to_testplan)

result = req_to_testplan.generate_consolidated_test_plan(
    llm_clients,
    'claude',  # 'claude' or 'gemini' or 'gpt'
    llm_clients.logger,
    "checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250826_085415.md",
    "../us-core/test_set/CapabilityStatement-us-core-server.html",
    "US Core IG",
    output_dir='checkpoints/testplan_generation/us-core'
)

print(f"Generated test plan with improved capability parsing: {result['test_plan_path']}")

INFO:root:Prompt environment set up at: /Users/ceadams/Documents/onclaive/onclaive/prompts
INFO:llm_utils:Starting test plan generation with claude for US Core IG
INFO:llm_utils:Parsed 66 requirements from checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250826_085415.md
INFO:llm_utils:Parsed capability statement from ../us-core/test_set/CapabilityStatement-us-core-server.html
INFO:llm_utils:Identifying group for requirement REQ-01 using claude...
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:llm_utils:Identifying group for requirement REQ-02 using claude...
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:llm_utils:Identifying group for requirement REQ-03 using claude...
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:llm_utils:Identifying group for requirement REQ-04 using claude...
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messag

Generated test plan with improved capability parsing: checkpoints/testplan_generation/us-core/claude_test_plan_20250826_230821.md


##### With RAG

In [33]:
import warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Set logging level to reduce noise
import logging
logging.getLogger("urllib3.connectionpool").setLevel(logging.ERROR)
logging.getLogger("backoff").setLevel(logging.ERROR)

import req_to_testplan_rag #import test plan generation script as module
importlib.reload(req_to_testplan_rag)

#clearing any existing capability statements from vector database
req_to_testplan_rag.clear_capability_collection("capability_statements")

Deleted existing collection: capability_statements


ERROR:backoff:Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)'))))


In [34]:
req_to_testplan_rag.generate_consolidated_test_plan(
    llm_clients, 
    'claude', #api
    llm_clients.logger, 
    "/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_refined_requirements_20250829_condition_profile.md", #input requirements list markdown file
    "checkpoints/markdown2/CapabilityStatement-us-core-server.md", #capability statement
    "US Core IG", #name of IG
    output_dir='checkpoints/testplan_generation/', #output directory
    verbose=True)

ERROR:backoff:Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)'))))
ERROR:backoff:Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)'))))



Processing Group: Condition (5 requirements)

Processing REQ-018: US Core Server SHALL support search-type and read for Condition

RAG RETRIEVAL FOR REQ-018
Query: US Core Server SHALL support search-type and read for Condition "**SHALL** support `search-type`, `read`." Condition Profile Interaction Summary for US Core Server US Core Server FHIR SHALL
Searching for 2 most relevant capability chunks...

Found 2 matching chunks:

  Match 1 (distance: 0.719982385635376):
  Length: 1452 chars
  Preview: FHIR Major Section: ## 14.3 CapabilityStatement: US Core Server CapabilityStatement

|  |  |  |  |  ...
  ...pi.json) | [Download](us-core-server.openapi.json)

  Match 2 (distance: 0.9184955358505249):
  Length: 2229 chars
  Preview: FHIR Resource/Component: ### 14.3.2 FHIR RESTful Capabilities

The US Core Server **SHALL**:

1. Sup...
  ... **MAY** support the `history-system` interaction.

Sending request to CLAUDE API...


ERROR:backoff:Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)'))))


Completed test specification for REQ-018

Processing REQ-019: US Core Server SHALL support patient search parameter for Condition

RAG RETRIEVAL FOR REQ-019
Query: US Core Server SHALL support patient search parameter for Condition "**SHALL** [patient](SearchParameter-us-core-condition-patient.html) reference" Condition Search Parameter Summary for US Core Server US Core Server FHIR SHALL
Searching for 2 most relevant capability chunks...

Found 2 matching chunks:

  Match 1 (distance: 0.6390321254730225):
  Length: 1452 chars
  Preview: FHIR Major Section: ## 14.3 CapabilityStatement: US Core Server CapabilityStatement

|  |  |  |  |  ...
  ...pi.json) | [Download](us-core-server.openapi.json)

  Match 2 (distance: 0.8115702271461487):
  Length: 90628 chars
  Preview: FHIR Resource Detail: #### 14.3.3.2 AllergyIntolerance

Conformance Expectation: **SHALL**

Supporte...
  ...d read/write formats for notes on the server.

---

Sending request to CLAUDE API...
Completed test specificati

{'requirements_count': 5,
 'group_count': 1,
 'test_plan_path': 'checkpoints/testplan_generation/claude_test_plan_20250829_072028.md'}

## Stage 4: Test Kit Generation
- Converts test specifications into executable Inferno Ruby tests
- Generates complete test suites with proper file organization
- Creates modular test structures following Inferno framework patterns
- Includes validation and alignment checking

Inputs: Test plan specification in markdown format

Outputs: Complete Inferno test kit

In [35]:
import plan_to_tests
import importlib
importlib.reload(plan_to_tests)

plan_to_tests.generate_inferno_test_kit(
    llm_clients, #initialize llm clients
    'claude',  #set API
    '/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/testplan_generation/claude_test_plan_20250829_072028.md',  #input test plan file
    module_name='US Core',
    output_dir='checkpoints/testkit_generation/us_core/test',  #output directory
    expected_actors=["Server", "Client"] #not used right now
)

Found 5 requirements with updated pattern
Processing requirement: REQ-018
Added requirement REQ-018 to section Condition
Processing requirement: REQ-019
Added requirement REQ-019 to section Condition
Processing requirement: REQ-020
Added requirement REQ-020 to section Condition
Processing requirement: REQ-021
Added requirement REQ-021 to section Condition
Processing requirement: REQ-022
Added requirement REQ-022 to section Condition
Final sections: ['Condition']
  Condition: 5 requirements


{'total_sections': 1,
 'total_requirements': 5,
 'generated_tests': 5,
 'module_dir': 'checkpoints/testkit_generation/us_core/test/claude_testkit_20250829_072456/us_core',
 'module_file': 'checkpoints/testkit_generation/us_core/test/claude_testkit_20250829_072456/claude_us_core_20250829_072456.rb',
 'output_dir': 'checkpoints/testkit_generation/us_core/test/claude_testkit_20250829_072456',
 'timestamp': '20250829_072456'}

In [37]:
# Save the script as recreate_module_file.py and run:
from recreate_module_file import recreate_module_file

# Adjust the path to match your actual output directory
output_dir = "/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/testkit_generation/us_core/gpt_testkit_20250828_103758"  # or whatever your actual path is

try:
    complete_module_path = recreate_module_file(output_dir, "US Core")
    print(f"Complete module file created at: {complete_module_path}")
except Exception as e:
    print(f"Error: {str(e)}")

Scanning test files in: /Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/testkit_generation/us_core/gpt_testkit_20250828_103758/us_core
Found 1136 test files
Test files by directory:
  allergyintolerance: 19 files
  applications_processing_documentreference_resources: 1 files
  applications_processing_us_core_medication_resources: 1 files
  applications_processing_us_core_medicationdispense__shall__always: 1 files
  applications_processing_us_core_medicationrequest: 1 files
  applications_processing_us_core_medicationrequest_dispenserequest: 1 files
  applications_processing_us_core_medicationrequest_dosageinstruction: 1 files
  capability_statement: 28 files
  careplan: 38 files
  careteam: 10 files
  certifying_client_application: 1 files
  certifying_client_applications: 2 files
  certifying_server_systems: 1 files
  certifying_system: 4 files
  certifying_systems: 3 files
  client: 5 files
  client_application: 9 files
  client_invoking_us_core_fetch_documentreferenc